In [1]:
import os, gc
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

VER = 1

In [2]:
df = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/train.csv')
TARGETS = df.columns[-6:]
print('Train shape:', df.shape )
print('Targets', list(TARGETS))
df.head()

Train shape: (106800, 15)
Targets ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']


,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,3,0,0,0,0,0
1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,3,0,0,0,0,0
2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,3,0,0,0,0,0
3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,3,0,0,0,0,0
4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,3,0,0,0,0,0


In [3]:
train = df.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg(
    {'spectrogram_id':'first','spectrogram_label_offset_seconds':'min'})
train.columns = ['spec_id','min']

tmp = df.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg(
    {'spectrogram_label_offset_seconds':'max'})
train['max'] = tmp

tmp = df.groupby('eeg_id')[['patient_id']].agg('first')
train['patient_id'] = tmp

tmp = df.groupby('eeg_id')[TARGETS].agg('sum')
for t in TARGETS:
    train[t] = tmp[t].values
    
y_data = train[TARGETS].values
y_data = y_data / y_data.sum(axis=1,keepdims=True)
train[TARGETS] = y_data

tmp = df.groupby('eeg_id')[['expert_consensus']].agg('first')
train['target'] = tmp

train = train.reset_index()
print('Train non-overlapp eeg_id shape:', train.shape )
train.head()

Train non-overlapp eeg_id shape: (17089, 12)


,eeg_id,spec_id,min,max,patient_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,target
0,568657,789577333,0.0,16.0,20654,0.0,0.000000,0.25,0.000000,0.166667,0.583333,Other
1,582999,1552638400,0.0,38.0,20230,0.0,0.857143,0.00,0.071429,0.000000,0.071429,LPD
2,642382,14960202,1008.0,1032.0,5955,0.0,0.000000,0.00,0.000000,0.000000,1.000000,Other
3,751790,618728447,908.0,908.0,38549,0.0,0.000000,1.00,0.000000,0.000000,0.000000,GPD
4,778705,52296320,0.0,0.0,40955,0.0,0.000000,0.00,0.000000,0.000000,1.000000,Other


In [4]:
def max_column_names(row):
    max_value = row.max()
    max_columns = row.index[row == max_value].nunique()
    return max_columns

train['max_cols_num'] = train[TARGETS].apply(lambda row: max_column_names(row), axis=1)

def max_column_name(row):
    max_col = row.idxmax()
    return max_col[:-5]


# apply関数を使用して各行に適用
train['max_column'] = train[TARGETS].apply(max_column_name, axis=1)
col_dict = {'seizure':'Seizure', 'gpd':'GPD', 'lrda':'LRDA', 'grda':'GRDA', 'lpd':'LPD', 'other':'Other'}
train['max_column'] = train['max_column'].map(col_dict)

In [5]:
# 最大投票率が複数のターゲットで重複している場合、モデルへの入力として有用か微妙なのでとりあえず省く。
# 観測の途中で明確に分類できるようになったパターンも混ざっているかもしれないので、要調査。

train[train['target']  != train['max_column']].sample(4)

,eeg_id,spec_id,min,max,patient_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,target,max_cols_num,max_column
12845,3216469722,1841034439,1246.0,1432.0,32481,0.015385,0.467308,0.000000,0.350000,0.0,0.167308,LRDA,1,LPD
5905,1469171274,1382475413,0.0,36.0,48272,0.000000,0.000000,0.333333,0.000000,0.0,0.666667,GPD,1,Other
17033,4279656286,319287046,488.0,562.0,26349,0.035088,0.000000,0.000000,0.929825,0.0,0.035088,Seizure,1,LRDA
9283,2310157379,747230293,1861.0,1927.0,35627,0.174603,0.095238,0.000000,0.539683,0.0,0.190476,Seizure,1,LRDA


In [6]:
df[df.eeg_id == 358676549]

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
28816,358676549,0,0.0,556413443,40,1924.0,3032166307,29441,GPD,0,3,5,2,0,3
28817,358676549,1,2.0,556413443,41,1926.0,3474848354,29441,GPD,0,3,5,2,0,3
28818,358676549,2,4.0,556413443,42,1928.0,2734871441,29441,GPD,0,3,5,2,0,3
28819,358676549,3,6.0,556413443,43,1930.0,649028773,29441,GPD,0,3,5,2,0,3
28820,358676549,4,36.0,556413443,44,1960.0,3697405091,29441,Other,0,0,1,0,0,2
28821,358676549,5,38.0,556413443,45,1962.0,1718400451,29441,Other,0,0,1,0,0,2
28822,358676549,6,56.0,556413443,46,1980.0,2321543874,29441,Other,0,0,1,0,0,2
28823,358676549,7,70.0,556413443,47,1994.0,2194447392,29441,Other,0,0,1,0,0,2
28824,358676549,8,74.0,556413443,48,1998.0,615312268,29441,Other,0,0,1,0,0,2
28825,358676549,9,78.0,556413443,49,2002.0,2636507426,29441,Other,0,0,1,0,0,2


In [7]:
READ_SPEC_FILES = False
READ_EEG_SPEC_FILES = False

In [8]:
%%time
# READ ALL SPECTROGRAMS
PATH = '/kaggle/input/hms-harmful-brain-activity-classification/train_spectrograms/'
files = os.listdir(PATH)
print(f'There are {len(files)} spectrogram parquets')

if READ_SPEC_FILES:    
    spectrograms = {}
    for i,f in enumerate(files):
        if i%100==0: print(i,', ',end='')
        tmp = pd.read_parquet(f'{PATH}{f}')
        name = int(f.split('.')[0])
        spectrograms[name] = tmp.iloc[:,1:].values
else:
    spectrograms = np.load('/kaggle/input/brain-spectrograms/specs.npy',allow_pickle=True).item()

There are 11138 spectrogram parquets
CPU times: user 4.27 s, sys: 9.27 s, total: 13.5 s
Wall time: 1min 39s


In [9]:
%%time
# READ ALL EEG SPECTROGRAMS
if READ_EEG_SPEC_FILES:
    all_eegs = {}
    for i,e in enumerate(train.eeg_id.values):
        if i%100==0: print(i,', ',end='')
        x = np.load(f'/kaggle/input/brain-eeg-spectrograms/EEG_Spectrograms/{e}.npy')
        all_eegs[e] = x
else:
    all_eegs = np.load('/kaggle/input/brain-eeg-spectrograms/eeg_specs.npy',allow_pickle=True).item()

CPU times: user 7.82 s, sys: 11.8 s, total: 19.6 s
Wall time: 1min 56s


In [10]:
%time
# ENGINEER FEATURES
import warnings
warnings.filterwarnings('ignore')

# FEATURE NAMES
SPEC_COLS = pd.read_parquet(f'{PATH}1000086677.parquet').columns[1:]
FEATURES = [f'{c}_mean_10m' for c in SPEC_COLS]
FEATURES += [f'{c}_min_10m' for c in SPEC_COLS]
FEATURES += [f'{c}_mean_20s' for c in SPEC_COLS]
FEATURES += [f'{c}_min_20s' for c in SPEC_COLS]
FEATURES += [f'eeg_mean_f{x}_10s' for x in range(512)]
FEATURES += [f'eeg_min_f{x}_10s' for x in range(512)]
FEATURES += [f'eeg_max_f{x}_10s' for x in range(512)]
FEATURES += [f'eeg_std_f{x}_10s' for x in range(512)]
print(f'We are creating {len(FEATURES)} features for {len(train)} rows... ',end='')

data = np.zeros((len(train),len(FEATURES)))
for k in range(len(train)):
    if k%100==0: print(k,', ',end='')
    row = train.iloc[k]
    r = int( (row['min'] + row['max'])//4 ) 

    # 10 MINUTE WINDOW FEATURES (MEANS and MINS)
    x = np.nanmean(spectrograms[row.spec_id][r:r+300,:],axis=0)
    data[k,:400] = x
    x = np.nanmin(spectrograms[row.spec_id][r:r+300,:],axis=0)
    data[k,400:800] = x

    # 20 SECOND WINDOW FEATURES (MEANS and MINS)
    x = np.nanmean(spectrograms[row.spec_id][r+145:r+155,:],axis=0)
    data[k,800:1200] = x
    x = np.nanmin(spectrograms[row.spec_id][r+145:r+155,:],axis=0)
    data[k,1200:1600] = x

    # RESHAPE EEG SPECTROGRAMS 128x256x4 => 512x256
    eeg_spec = np.zeros((512,256),dtype='float32')
    xx = all_eegs[row.eeg_id]
    for j in range(4): eeg_spec[128*j:128*(j+1),] = xx[:,:,j]

    # 10 SECOND WINDOW FROM EEG SPECTROGRAMS 
    x = np.nanmean(eeg_spec.T[100:-100,:],axis=0)
    data[k,1600:2112] = x
    x = np.nanmin(eeg_spec.T[100:-100,:],axis=0)
    data[k,2112:2624] = x
    x = np.nanmax(eeg_spec.T[100:-100,:],axis=0)
    data[k,2624:3136] = x
    x = np.nanstd(eeg_spec.T[100:-100,:],axis=0)
    data[k,3136:3648] = x

train[FEATURES] = data
print(); print('New train shape:',train.shape)

CPU times: user 2 µs, sys: 2 µs, total: 4 µs
Wall time: 9.06 µs
We are creating 3648 features for 17089 rows... 0 , 100 , 200 , 300 , 400 , 500 , 600 , 700 , 800 , 900 , 1000 , 1100 , 1200 , 1300 , 1400 , 1500 , 1600 , 1700 , 1800 , 1900 , 2000 , 2100 , 2200 , 2300 , 2400 , 2500 , 2600 , 2700 , 2800 , 2900 , 3000 , 3100 , 3200 , 3300 , 3400 , 3500 , 3600 , 3700 , 3800 , 3900 , 4000 , 4100 , 4200 , 4300 , 4400 , 4500 , 4600 , 4700 , 4800 , 4900 , 5000 , 5100 , 5200 , 5300 , 5400 , 5500 , 5600 , 5700 , 5800 , 5900 , 6000 , 6100 , 6200 , 6300 , 6400 , 6500 , 6600 , 6700 , 6800 , 6900 , 7000 , 7100 , 7200 , 7300 , 7400 , 7500 , 7600 , 7700 , 7800 , 7900 , 8000 , 8100 , 8200 , 8300 , 8400 , 8500 , 8600 , 8700 , 8800 , 8900 , 9000 , 9100 , 9200 , 9300 , 9400 , 9500 , 9600 , 9700 , 9800 , 9900 , 10000 , 10100 , 10200 , 10300 , 10400 , 10500 , 10600 , 10700 , 10800 , 10900 , 11000 , 11100 , 11200 , 11300 , 11400 , 11500 , 11600 , 11700 , 11800 , 11900 , 12000 , 12100 , 12200 , 12300 , 12400 , 

del all_eegs, spectrograms, data
gc.collect()

In [11]:
del all_eegs, spectrograms, data
gc.collect()

0

In [12]:
import lightgbm as lgb
from sklearn.metrics import log_loss
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold, GroupKFold

params = {
    'objective': 'multiclass',
    'num_class': 6,
    'metric': 'multi_logloss',
    'n_estimators': 2500,  # 大きな値で設定しておき、早期停止で終了する
    'learning_rate': 0.01,
#    'device': 'gpu',  # GPUを使用する場合の設定
#     'max_depth': 15,  # 木の深さ
#     'num_leaves' : 100,
    'importance_type': 'gain',
    'verbose' : 1,
}

# 早期停止用のコールバック設定
early_stopping_rounds = 100  # 100イテレーション連続で改善が見られない場合に停止
callbacks = [lgb.early_stopping(early_stopping_rounds, first_metric_only=True)]

all_oof = []
all_true = []
TARS = {'Seizure':0, 'LPD':1, 'GPD':2, 'LRDA':3, 'GRDA':4, 'Other':5}

gkf = GroupKFold(n_splits=5)
for i, (train_index, valid_index) in enumerate(gkf.split(train, train.target, train.patient_id)):   
    print('#'*25)
    print(f'### Fold {i+1}')
    print(f'### train size {len(train_index)}, valid size {len(valid_index)}')
    print('#'*25)
    
    model = LGBMClassifier(**params, n_jobs=-1)
    
    X_train = train.loc[train_index, FEATURES]
    y_train = train.loc[train_index,'target'].map(TARS)
    
    X_valid = train.loc[valid_index, FEATURES]
    y_valid = train.loc[valid_index,'target'].map(TARS)
    
    
    model.fit(
        X_train, y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        eval_names=['train', 'valid'],
        eval_metric='multi_logloss',
        callbacks=callbacks,
    )
    
    best_iteration = model.best_iteration_
    model.booster_.save_model(f'lgb_model_fold{i}.h5', num_iteration=best_iteration)
    
    oof = model.predict_proba(X_valid)
    all_oof.append(oof)
    all_true.append(train.loc[valid_index, TARGETS].values)
    
    del X_train, X_valid ,y_train, y_valid
    
all_oof = np.concatenate(all_oof)
all_true = np.concatenate(all_true)

#########################
### Fold 1
### train size 13671, valid size 3418
#########################
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.128751 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 915349
[LightGBM] [Info] Number of data points in the train set: 13671, number of used features: 3648
[LightGBM] [Info] Start training from score -1.823190
[LightGBM] [Info] Start training from score -1.825911
[LightGBM] [Info] Start training from score -2.308528
[LightGBM] [Info] Start training from score -2.964834
[LightGBM] [Info] Start training from score -2.280234
[LightGBM] [Info] Start training from score -0.857591
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[505]	train's multi_logloss: 0.291719	valid's multi_logloss: 1.01475
Evaluated only: multi_logloss
#########################
### Fold 2
### train size 13671, valid size 3418
###########

In [13]:
import sys
sys.path.append('/kaggle/input/kaggle-kl-div')
from kaggle_kl_div import score

oof = pd.DataFrame(all_oof.copy())
oof['id'] = np.arange(len(oof))

true = pd.DataFrame(all_true.copy())
true['id'] = np.arange(len(true))

cv = score(solution=true, submission=oof, row_id_column_name='id')
print('CV Score KL-Div for EfficientNetB2 =',cv)

CV Score KL-Div for EfficientNetB2 = 0.7536374651793687
